## Binary Classification Model – M01 🌾

**Objective:**  
Model M01 is a binary classification model designed to detect whether a given wheat plant is **healthy** or **diseased** based on input images. This task is crucial for early identification and management of crop diseases to reduce yield losses.

**Role:**  
The model takes an image of a wheat crop and predicts one of two classes:
- **Healthy**
- **Diseased**

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201, EfficientNetB2, VGG19, DenseNet169

from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dropout, Dense, Multiply, Add
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

In [ ]:
# Hyperparameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 1e-6
PATIENCE = 2
FACTOR = 0.5
OPTIMIZER = Adam(learning_rate=LEARNING_RATE)
LOSS = "categorical_crossentropy"


data_dir = "./D01"
# Apply Gamma Correction
def apply_gamma_correction(image, gamma=1.5):
    image = (image * 255).astype("uint8")  # Convert to uint8
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    corrected_image = cv2.LUT(image, table)
    return corrected_image.astype("float32") / 255.0  # Convert back to float32 (0-1 range)

# Apply Cutout
def cutout(image, mask_size=50):
    h, w, _ = image.shape
    y, x = np.random.randint(h), np.random.randint(w)
    y1, y2 = np.clip([y - mask_size // 2, y + mask_size // 2], 0, h)
    x1, x2 = np.clip([x - mask_size // 2, x + mask_size // 2], 0, w)
    image[y1:y2, x1:x2, :] = 0
    return image

# Apply MixUp
def mixup(image1, image2, label1, label2, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    mixed_image = lam * image1 + (1 - lam) * image2
    mixed_label = lam * label1 + (1 - lam) * label2
    return mixed_image, mixed_label

# Apply CutMix
def cutmix(image1, image2, label1, label2):
    h, w, _ = image1.shape
    lam = np.random.beta(1.0, 1.0)
    cx, cy = np.random.randint(w), np.random.randint(h)
    bw, bh = int(w * np.sqrt(1 - lam)), int(h * np.sqrt(1 - lam))
    x1, y1 = max(cx - bw // 2, 0), max(cy - bh // 2, 0)
    x2, y2 = min(cx + bw // 2, w), min(cy + bh // 2, h)
    image1[y1:y2, x1:x2, :] = image2[y1:y2, x1:x2, :]
    lam = 1 - ((x2 - x1) * (y2 - y1) / (h * w))
    mixed_label = lam * label1 + (1 - lam) * label2
    return image1, mixed_label

# Load and augment dataset
def load_data_with_augmentations(data_dir):
    images, labels = [], []
    class_names = os.listdir(data_dir)

    for label, class_dir in enumerate(class_names):
        class_path = os.path.join(data_dir, class_dir)
        if not os.path.isdir(class_path):
            continue

        image_list = []
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, IMG_SIZE) / 255.0
            images.append(img)
            labels.append(label)
            image_list.append(img)

        # Apply Cutout & Gamma Correction
        for img in image_list:
            images.append(cutout(img.copy()))
            labels.append(label)
            images.append(apply_gamma_correction(img.copy()))
            labels.append(label)

    images = np.array(images)
    labels = to_categorical(labels, num_classes=len(class_names))

    # Apply MixUp and CutMix
    augmented_images, augmented_labels = [], []
    for i in range(len(images) - 1):
        img1, img2 = images[i], images[i + 1]
        lbl1, lbl2 = labels[i], labels[i + 1]
        mixup_img, mixup_lbl = mixup(img1, img2, lbl1, lbl2)
        cutmix_img, cutmix_lbl = cutmix(img1, img2, lbl1, lbl2)
        augmented_images.extend([mixup_img, cutmix_img])
        augmented_labels.extend([mixup_lbl, cutmix_lbl])

    augmented_images = np.array(augmented_images)
    augmented_labels = np.array(augmented_labels)

    images = np.concatenate([images, augmented_images], axis=0)
    labels = np.concatenate([labels, augmented_labels], axis=0)

    return images, labels

In [ ]:
# Load dataset with augmentations
X, y = load_data_with_augmentations(data_dir)

In [ ]:
from sklearn.model_selection import train_test_split

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D

def create_model(base_model, input_shape):
    base = base_model(weights="imagenet", include_top=False, input_tensor=Input(shape=input_shape))
    x = GlobalAveragePooling2D()(base.output) 
    return Model(inputs=base.input, outputs=x)



input_shape = (224, 224, 3)
model_DenseNet169 = create_model(DenseNet169, input_shape)
model_VGG19 = create_model(VGG19, input_shape)
model_EfficientNetB2 = create_model(EfficientNetB2, input_shape)



# Feature extraction in batches
def extract_features_in_batches(model, data, batch_size=32):
    features = []
    for start in range(0, len(data), batch_size):
        end = start + batch_size
        batch_data = data[start:end]
        batch_tensor = tf.convert_to_tensor(batch_data, dtype=tf.float32)
        batch_features = model.predict(batch_tensor)  
        features.append(batch_features)
    return np.vstack(features)


In [ ]:
features_DenseNet169 = extract_features_in_batches(model_DenseNet169, X_train, batch_size=BATCH_SIZE)
features_VGG19 = extract_features_in_batches(model_VGG19, X_train, batch_size=BATCH_SIZE)
features_EfficientNetB2 = extract_features_in_batches(model_EfficientNetB2, X_train, batch_size=BATCH_SIZE)

X_train_features = np.concatenate([features_DenseNet169, features_VGG19, features_EfficientNetB2], axis=1)

c:\Users\DELL\Documents\PFE\M1\.venv\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 911ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 895ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 861ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━

c:\Users\DELL\Documents\PFE\M1\.venv\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


c:\Users\DELL\Documents\PFE\M1\.venv\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_596']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


c:\Users\DELL\Documents\PFE\M1\.venv\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_619']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

c:\Users\DELL\Documents\PFE\M1\.venv\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_619']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [ ]:
features_DenseNet169_test = extract_features_in_batches(model_DenseNet169, X_test, batch_size=BATCH_SIZE)
features_VGG19_test = extract_features_in_batches(model_VGG19, X_test, batch_size=BATCH_SIZE)
features_EfficientNetB2_test = extract_features_in_batches(model_EfficientNetB2, X_test, batch_size=BATCH_SIZE)

X_test_features = np.concatenate([features_DenseNet169_test, features_VGG19_test, features_EfficientNetB2_test], axis=1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 972ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 883ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 895ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 894ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━

In [ ]:
# 1️⃣ Définir les hyperparamètres à tester pour chaque modèle
rf_params = {
    "n_estimators": [50, 100, 200],
    "max_features": ["sqrt", "log2"],
    "min_samples_leaf": [1, 2, 5],
    "min_samples_split": [2, 5, 10]
}

dt_params = {
    "max_depth": [50, 100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1, 2, 5]
}

mlp_params = {
    "hidden_layer_sizes": [(100, 50), (150, 100) ,(100,)],
    "activation": ["relu", "tanh"],
    "solver": ["adam", "sgd"],
    "learning_rate": ["constant", "adaptive"]
}

svm_params = {
    "C": [0.1, 1, 10],                    
    "kernel": ["linear", "rbf"],   
    "gamma": ["scale", "auto"],            
    "degree": [2, 3, 4],                   
}

In [ ]:
# Recherche des meilleurs hyperparamètres avec GridSearchCV
print("Optimisation du RandomForest...")
rf = GridSearchCV(RandomForestClassifier(class_weight="balanced", random_state=1), rf_params, cv=3, n_jobs=-1)
rf.fit(X_train_features, np.argmax(y_train, axis=1))
best_rf = rf.best_estimator_
print("Meilleurs paramètres RF:", rf.best_params_)

Optimisation du RandomForest...
Meilleurs paramètres RF: {'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


In [ ]:
#print("Optimisation du DecisionTree...")
dt = GridSearchCV(DecisionTreeClassifier(class_weight="balanced", random_state=0), dt_params, cv=3, n_jobs=-1)
dt.fit(X_train_features, np.argmax(y_train, axis=1))
best_dt = dt.best_estimator_
print("Meilleurs paramètres DT:", dt.best_params_) 

Meilleurs paramètres DT: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 1}


In [ ]:
from sklearn.svm import SVC
svm = GridSearchCV(SVC(class_weight="balanced", probability=True, random_state=0), svm_params, cv=3, n_jobs=-1)
svm.fit(X_train_features, np.argmax(y_train, axis=1))


AttributeError: 'GridSearchCV' object has no attribute 'best_estimavtor_'

In [ ]:
# Get the best estimator and print the best parameters
best_svm = svm.best_estimator_
print("Meilleurs paramètres SVM:", svm.best_params_)

Meilleurs paramètres SVM: {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
mlp = GridSearchCV(MLPClassifier(max_iter=600, random_state=42), mlp_params, cv=3, n_jobs=-1)
mlp.fit(X_train_features, np.argmax(y_train, axis=1))
best_mlp = mlp.best_estimator_
print("Meilleurs paramètres MLP:", mlp.best_params_)

Meilleurs paramètres MLP: {'activation': 'relu', 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant', 'solver': 'adam'}


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
import numpy as np


y_test_pred = best_mlp.predict(X_test_features)

y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_test_pred))

test_accuracy = accuracy_score(y_true, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

f1 = f1_score(y_true, y_test_pred, average='macro')
print(f"F1 Score (macro): {f1:.4f}")

recall = recall_score(y_true, y_test_pred, average='macro')
print(f"Recall (macro): {recall:.4f}")

precision = precision_score(y_true, y_test_pred, average='macro')
print(f"Precision (macro): {precision:.4f}")


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       670
           1       1.00      0.99      0.99       831

    accuracy                           0.99      1501
   macro avg       0.99      0.99      0.99      1501
weighted avg       0.99      0.99      0.99      1501

Test Accuracy: 0.9940
F1 Score (macro): 0.9939
Recall (macro): 0.9940
Precision (macro): 0.9939


In [ ]:
import joblib
joblib.dump(best_mlp, "M1_mlp.pkl")

['M1_mlp.pkl']

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
import numpy as np

# Predict test labels
y_test_pred = best_svm.predict(X_test_features)

# True labels
y_true = np.argmax(y_test, axis=1)

# Print classification report (includes precision, recall, f1-score per class)
print(classification_report(y_true, y_test_pred))

# Calculate and print test accuracy
test_accuracy = accuracy_score(y_true, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Calculate and print macro-averaged F1 score
f1 = f1_score(y_true, y_test_pred, average='macro')
print(f"F1 Score (macro): {f1:.4f}")

# Calculate and print macro-averaged recall
recall = recall_score(y_true, y_test_pred, average='macro')
print(f"Recall (macro): {recall:.4f}")

# Calculate and print macro-averaged precision
precision = precision_score(y_true, y_test_pred, average='macro')
print(f"Precision (macro): {precision:.4f}")


              precision    recall  f1-score   support

           0       1.00      0.99      1.00       670
           1       1.00      1.00      1.00       831

    accuracy                           1.00      1501
   macro avg       1.00      1.00      1.00      1501
weighted avg       1.00      1.00      1.00      1501

Test Accuracy: 0.9960
F1 Score (macro): 0.9960
Recall (macro): 0.9958
Precision (macro): 0.9961


In [ ]:
import joblib
joblib.dump(best_svm, "M1_svm.pkl")

['M1_svm.pkl']

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
import numpy as np

# Predict test labels
y_test_pred = best_dt.predict(X_test_features)

# True labels
y_true = np.argmax(y_test, axis=1)

# Print classification report (includes precision, recall, f1-score per class)
print(classification_report(y_true, y_test_pred))

# Calculate and print test accuracy
test_accuracy = accuracy_score(y_true, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Calculate and print macro-averaged F1 score
f1 = f1_score(y_true, y_test_pred, average='macro')
print(f"F1 Score (macro): {f1:.4f}")

# Calculate and print macro-averaged recall
recall = recall_score(y_true, y_test_pred, average='macro')
print(f"Recall (macro): {recall:.4f}")

# Calculate and print macro-averaged precision
precision = precision_score(y_true, y_test_pred, average='macro')
print(f"Precision (macro): {precision:.4f}")


              precision    recall  f1-score   support

           0       0.94      0.96      0.95       670
           1       0.97      0.95      0.96       831

    accuracy                           0.95      1501
   macro avg       0.95      0.96      0.95      1501
weighted avg       0.96      0.95      0.95      1501

Test Accuracy: 0.9547
F1 Score (macro): 0.9543
Recall (macro): 0.9553
Precision (macro): 0.9534


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
import numpy as np

# Predict test labels
y_test_pred = best_rf.predict(X_test_features)

# True labels
y_true = np.argmax(y_test, axis=1)

# Print classification report (includes precision, recall, f1-score per class)
print(classification_report(y_true, y_test_pred))

# Calculate and print test accuracy
test_accuracy = accuracy_score(y_true, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Calculate and print macro-averaged F1 score
f1 = f1_score(y_true, y_test_pred, average='macro')
print(f"F1 Score (macro): {f1:.4f}")

# Calculate and print macro-averaged recall
recall = recall_score(y_true, y_test_pred, average='macro')
print(f"Recall (macro): {recall:.4f}")

# Calculate and print macro-averaged precision
precision = precision_score(y_true, y_test_pred, average='macro')
print(f"Precision (macro): {precision:.4f}")


              precision    recall  f1-score   support

           0       1.00      0.97      0.98       670
           1       0.98      1.00      0.99       831

    accuracy                           0.99      1501
   macro avg       0.99      0.99      0.99      1501
weighted avg       0.99      0.99      0.99      1501

Test Accuracy: 0.9867
F1 Score (macro): 0.9865
Recall (macro): 0.9852
Precision (macro): 0.9880
